# An example notebook how to generate training data for YOLOv5 from an .EMObs file

In [1]:
import os, sys
import cv2

sys.path.append("..")

import emtmlibpy.emtmlibpy as emtm
from emtmlibpy.emtmlibpy import EMTMResult

import numpy as np
import pandas as pd
from collections import namedtuple

### Change this next cell to the path that has the .emobs and video files.

In [3]:
SRC_DIR = os.path.abspath(os.getcwd())
TEST_FILES_PATH = os.path.join(SRC_DIR, '..', 'test_files')

In [66]:
TEST_FILES_PATH = '/home/marrabld/data/test_emobs/emtmlibpy'
TEST_FILE = 'afid.EMObs'

### Sanity check that we can read the library

In [68]:
print(emtm.emtm_version())

(2, 0)


### Load the .EMObs files

In [69]:
r = emtm.em_load_data(os.path.join(TEST_FILES_PATH, TEST_FILE))
assert EMTMResult(r) == EMTMResult(0)

Load the data in Test.EMObs into a dataframe

In [70]:
# this needs to be called first to generate the list
n_fgs = emtm.em_unique_fgs()
fgs = []

for ii in range(n_fgs):
    fgs.append(emtm.em_get_unique_fgs(ii))

print(fgs)

[(b'', b'', b''), (b'lethrinidae', b'lethrinus', b'punctulatus'), (b'lutjanidae', b'lutjanus', b'sebae')]


### Write the classes out to file

In [71]:
with open('./train/classes.txt', 'w') as f:
    for _class in fgs:
        f.write(f"{_class[0].decode('utf-8')}_{_class[1].decode('utf-8')}_{_class[2].decode('utf-8')}\n")

In [72]:
def point_to_df():
    """
    Convert EvemtMeasure points to a dataframe
    :return: Pandas dataframe
    """
    dtype_template = 'object'
    point_count, box_count = emtm.em_point_count()
    print(f'number of points :: {point_count}')
    print(f'number of boxes :: {box_count}')

    p = emtm.em_get_point(0)
    index = [attr for attr in dir(p) if (not attr.startswith('__') and not attr.startswith('_'))]
    data = np.empty(shape=[point_count, len(index)], dtype=dtype_template)  # change these

    for jj in range(point_count):
        p = emtm.em_get_point(jj)
        for ii, ind in enumerate(index):
            tmp = p.__getattribute__(ind)
            data[jj][ii] = tmp

    xpdf = pd.DataFrame(data=data, columns=index)
    xpdf = xpdf.convert_dtypes().infer_objects()
    return xpdf

In [73]:
pdf = point_to_df()
pdf.head()

number of points :: 237
number of boxes :: 237


,d_imx,d_imy,d_period_time_mins,d_rectx,d_recty,d_time_mins,n_frame,str_activity,str_att_10,str_att_9,str_code,str_comment,str_family,str_filename,str_genus,str_number,str_op_code,str_period,str_species,str_stage
0,767,420,-1,108,77,3.868865,6957,b'Passing',b'',b'',b'',b'',b'',b'G000048 L_small.m4v',b'',b'1',b'AFID',b'',b'',b'AD'
1,374,579,-1,290,137,3.868865,6957,b'Passing',b'',b'',b'',b'',b'',b'G000048 L_small.m4v',b'',b'1',b'AFID',b'',b'',b'AD'
2,191,407,-1,65,27,3.868865,6957,b'Passing',b'',b'',b'',b'',b'',b'G000048 L_small.m4v',b'',b'1',b'AFID',b'',b'',b'AD'
3,659,388,-1,27,21,3.868865,6957,b'Passing',b'',b'',b'',b'',b'',b'G000048 L_small.m4v',b'',b'1',b'AFID',b'',b'',b'AD'
4,648,533,-1,360,145,3.868865,6957,b'Passing',b'',b'',b'',b'',b'',b'G000048 L_small.m4v',b'',b'1',b'AFID',b'',b'',b'AD'


In [74]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   d_imx               237 non-null    Int64  
 1   d_imy               237 non-null    Int64  
 2   d_period_time_mins  237 non-null    Int64  
 3   d_rectx             237 non-null    Int64  
 4   d_recty             237 non-null    Int64  
 5   d_time_mins         237 non-null    Float64
 6   n_frame             237 non-null    Int64  
 7   str_activity        237 non-null    object 
 8   str_att_10          237 non-null    object 
 9   str_att_9           237 non-null    object 
 10  str_code            237 non-null    object 
 11  str_comment         237 non-null    object 
 12  str_family          237 non-null    object 
 13  str_filename        237 non-null    object 
 14  str_genus           237 non-null    object 
 15  str_number          237 non-null    object 
 16  str_op_c

### We're only interested in boxes at this stage so subset the data frame to include only boxes

In [75]:
boxdf = pdf[pdf['d_rectx'] >= 0]
boxdf.head()

,d_imx,d_imy,d_period_time_mins,d_rectx,d_recty,d_time_mins,n_frame,str_activity,str_att_10,str_att_9,str_code,str_comment,str_family,str_filename,str_genus,str_number,str_op_code,str_period,str_species,str_stage
0,767,420,-1,108,77,3.868865,6957,b'Passing',b'',b'',b'',b'',b'',b'G000048 L_small.m4v',b'',b'1',b'AFID',b'',b'',b'AD'
1,374,579,-1,290,137,3.868865,6957,b'Passing',b'',b'',b'',b'',b'',b'G000048 L_small.m4v',b'',b'1',b'AFID',b'',b'',b'AD'
2,191,407,-1,65,27,3.868865,6957,b'Passing',b'',b'',b'',b'',b'',b'G000048 L_small.m4v',b'',b'1',b'AFID',b'',b'',b'AD'
3,659,388,-1,27,21,3.868865,6957,b'Passing',b'',b'',b'',b'',b'',b'G000048 L_small.m4v',b'',b'1',b'AFID',b'',b'',b'AD'
4,648,533,-1,360,145,3.868865,6957,b'Passing',b'',b'',b'',b'',b'',b'G000048 L_small.m4v',b'',b'1',b'AFID',b'',b'',b'AD'


In [76]:
def em_box_coords_to_yolo(x: float, y: float, box_width: float, box_height: float, image_width: float=1920, image_height: float=1080) -> namedtuple:
    """
    Helper function to transform the EventMeasure coord to yolo coords.
    YOLO coordinates are x, y, centre of the box and all coords normalised by image width and height
    :param image_height:
    :param image_width:
    :param box_height:
    :param box_width:
    :param x: Pixel column
    :param y: Pixel row
    :return: namedtuple (x, y, width, height)
    """

    XYWH = namedtuple('XYHW', 'x y width height')

    yolo_x = (x + box_width / 2) / image_width
    yolo_y = (y + box_height / 2) / image_height
    yolo_box_width = box_width / image_width
    yolo_box_height = box_height / image_height

    return XYWH(yolo_x, yolo_y, yolo_box_width, yolo_box_height)

In [77]:
def df_to_yolo(df: pd.DataFrame, out_dir='train', image_width=1920, image_height=1080):
    """
    Given a data frame write out a directory with training data in the YOLO format
    <object-class> <x> <y> <width> <height>

    integer number of object from 0 to (classes-1) - float values relative to width and height of image, it can be equal from (0.0 to 1.0] for example: <x> = <absolute_x> / <image_width> or <height> = <absolute_height> / <image_height> atention: <x> <y> - are center of rectangle (are not top-left corner)
    :param df:
    :return:
    """
    os.makedirs(out_dir, exist_ok=True)


    label = []
    for index, row in df.iterrows():
        #concatinate fgs for the label name
        label.append(f"{row['str_family'].decode('utf-8')}_{row['str_genus'].decode('utf-8')}_{row['str_species'].decode('utf-8')}")
    label = list(set(label))
    print(f'Unique Labels :: {label}')

    # Now we have the unique labels pull them out and create the string format.

    for index, row in df.iterrows():
        with open(os.path.join(out_dir, f"{df['str_filename'].iloc[index].decode('utf-8')}_{df['n_frame'].iloc[index]}.txt"), 'ab') as f:
            row_label = []
            row_string = []
            r = em_box_coords_to_yolo(row['d_imx'], row['d_imy'], row['d_rectx'], row['d_recty'])
            #concatinate fgs for the label name
            row_label = f"{row['str_family'].decode('utf-8')}_{row['str_genus'].decode('utf-8')}_{row['str_species'].decode('utf-8')}"
            label_num = label.index(row_label)
            # Add the coordinates and boxes
            row_string.append(f"{label_num} {r.x} {r.y} {r.width} {r.height}")

            print(f'Writing :: {row_string}')
            f.write(f'{row_string[0]}\n'.encode())

In [78]:
df_to_yolo(boxdf)

Unique Labels :: ['Lutjanidae_Lutjanus_sebae', '__', 'Lethrinidae_Lethrinus_punctulatus']
Writing :: ['1 0.4276041666666667 0.42453703703703705 0.05625 0.0712962962962963']
Writing :: ['1 0.2703125 0.5995370370370371 0.15104166666666666 0.12685185185185185']
Writing :: ['1 0.11640625 0.38935185185185184 0.033854166666666664 0.025']
Writing :: ['1 0.3502604166666667 0.36898148148148147 0.0140625 0.019444444444444445']
Writing :: ['1 0.43125 0.5606481481481481 0.1875 0.13425925925925927']
Writing :: ['1 0.15260416666666668 0.4583333333333333 0.028125 0.040740740740740744']
Writing :: ['1 0.7776041666666667 0.4708333333333333 0.027083333333333334 0.030555555555555555']
Writing :: ['1 0.40859375 0.42268518518518516 0.06927083333333334 0.0712962962962963']
Writing :: ['1 0.14583333333333334 0.38796296296296295 0.034375 0.024074074074074074']
Writing :: ['1 0.453125 0.5569444444444445 0.16770833333333332 0.12129629629629629']
Writing :: ['1 0.27838541666666666 0.6018518518518519 0.1484375 0.

In [85]:
def extract_frames_from_video(video, frame_number):
    """
    Given a BRUVS video, extract the frame as a jpg.
    :param video:
    :param frame_number:
    :return:
    """
    vid = cv2.VideoCapture(video)
    vid.set(1, frame_number)

    ret, image = vid.read()
    return image

In [80]:
img = extract_frames_from_video(os.path.join(TEST_FILES_PATH,boxdf['str_filename'].iloc[0].decode('utf-8')), boxdf['n_frame'].iloc[0])

In [81]:
cv2.imwrite('test.png', img)

True

<img src='test.png'>


In [82]:
boxdf['n_frame'].unique()

<IntegerArray>
[ 6957,  6965,  6992,  7012,  7020,  7035,  7338,  7365,  7571,  7577,  7628,
 10796, 10913, 10916, 11623, 14521, 17780, 17786, 17807, 17981]
Length: 20, dtype: Int64

In [84]:
for ii, frame in enumerate(boxdf['n_frame'].unique()):
    print(frame)
    ii_loc = pd.Index(boxdf['n_frame']).get_loc(frame).start # slice object
    print(ii_loc)
    img = extract_frames_from_video(os.path.join(TEST_FILES_PATH, boxdf['str_filename'].iloc[ii_loc].decode('utf-8')), boxdf['n_frame'].iloc[ii_loc])
    cv2.imwrite(f"train/{boxdf['str_filename'].iloc[ii_loc].decode('utf-8')}_{boxdf['n_frame'].iloc[ii_loc]}.png", img)

6957
0
6965
7
6992
11
7012
19


KeyboardInterrupt: 